<a href="https://colab.research.google.com/github/Mubashar-Bashir/OpenAI-SDK/blob/main/context_openai_agents_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install openai-agents SDK

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


# Make your Notebook capable of running asynchronous functions.
Both Jupyter notebooks and Python’s asyncio library utilize event loops, but they serve different purposes and can sometimes interfere with each other.

The nest_asyncio library allows the existing event loop to accept nested event loops, enabling asyncio code to run within environments that already have an event loop, such as Jupyter notebooks.

In summary, both Jupyter notebooks and Python’s asyncio library utilize event loops to manage asynchronous operations. When working within Jupyter notebooks, it’s essential to be aware of the existing event loop to effectively run asyncio code without conflicts.

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Config

In [3]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)
from google.colab import userdata


In [4]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [5]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

# Context Management


Context available locally to your code: this is data and dependencies you might need when tool functions run, during callbacks like on_handoff, in lifecycle hooks, etc.
- You create any Python object you want. A common pattern is to use a dataclass or a Pydantic object.
- You pass that object to the various run methods (e.g. Runner.run(..., **context=whatever**)).
- All your tool calls, lifecycle hooks etc will be passed a wrapper object, - RunContextWrapper[T], where T represents your context object type which you can access via wrapper.context.
- The most important thing to be aware of: every agent, tool function, lifecycle etc for a given agent run must use the same type of context.

You can use the context for things like:
- Contextual data for your run (e.g. things like a username/uid or other information about the user)
- Dependencies (e.g. logger objects, data fetchers, etc)
- Helper functions

[Learning Reference](https://openai.github.io/openai-agents-python/context/)

# Local Context

In [13]:
from agents import enable_verbose_stdout_logging, set_tracing_disabled
set_tracing_disabled(True)
# enable_verbose_stdout_logging()

In [23]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo1:
    name: str
    uid: int
    cell:str | None
    Address: str|None
    Account_Number:str = "ALF12345563"
    location: str = "Pakistan"
    balance: float = 0.0


@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the age of the user.'''
    print("wrapper User Age >>> ",wrapper)
    return f"User {wrapper.context.name} is 30 years old"

@function_tool
async def fetch_user_location(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the location of the user.'''
    print("wrapper Location >>> ",wrapper)
    return f"User {wrapper.context.name} is from {wrapper.context.location}"
#to add for user payment to his account
@function_tool
async def user_payment(wrapper: RunContextWrapper[UserInfo1], amount: float) -> str:
    '''Adds the specified amount to the user's account balance.'''
    print("wrapper Payment >>> ",wrapper.context.balance)
    print("Amount recived>>>", amount)

    account_number=wrapper.context.Account_Number
    print(f" Account number = {wrapper.context.Account_Number}")
    Balance=wrapper.context.balance + float(amount)
    wrapper.context.balance = Balance
    return f"User Balance# {wrapper.context.balance}"
# show the balance Amount of user
@function_tool
async def check_balance(wrapper: RunContextWrapper[UserInfo1]) -> str:
    '''Returns the current account balance of the user.'''
    return f"User {wrapper.context.name}'s account balance is {wrapper.context.balance} Rupees."
#create

async def main():
    user_info = UserInfo1(name="Muhammad Qasim", uid=123, location="KHI", cell="03446062616",Address="GRW")

    agent = Agent[UserInfo1](
        name="Assistant",
        instructions="Assistant can help user to add Balance in the said Account",
        tools=[fetch_user_age,fetch_user_location,user_payment,check_balance],
        model=model
    )

    result = await Runner.run(
        starting_agent=agent,
        input="Add 15Lac Rupees to the Muhammad Qasim Account, check the Balance",
        # input="which tool are you using for adding amount into user account , and tell me the account nUmber",
        context=user_info,
    )

    print(result.final_output)
    # The user John is 47 years old.

if __name__ == "__main__":
    asyncio.run(main())

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7d372b4b4cb0>


...

LLM resp:
{
  "content": "OK. I've added 1500000.0 Rupees to the Muhammad Qasim Account. The current balance is 1500000.0 Rupees.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "OK. I've added 1500000.0 Rupees to the Muhammad Qasim Account. The current balance is 1500000.0 Rupees.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


OK. I've added 1500000.0 Rupees to the Muhammad Qasim Account. The current balance is 1500000.0 Rupees.



# Agent/LLM context

In [11]:
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

In [24]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:
    name: str
    uid: int
    cell:str|None
    Address: str|None
    Account_Number:str|None

@function_tool
async def greet_user(context: RunContextWrapper[UserInfo], greeting: str) -> str:
  """Greets the User with their name.
  Args:
    greeting: A specialed greeting message for user
  """
  name = context.context.name
  return f"Hello {name}, {greeting}"

async def main():
    user_info = UserInfo(name="Mubashar", uid=123, cell="0300XXXX", Address="Pakistan", Account_Number="3435654")

    agent = Agent[UserInfo](
        name="Assistant",
        tools=[greet_user],
        model=model,
        # Dynamic function context pass in instructions/system prompmt/developer prompt
        instructions="Always greet the user using <function_call>greet_user</function_call> and welcome them to Panaversity"
    )

    result = await Runner.run(
        starting_agent=agent,
        input="Hello",
        context=user_info,
    )

    print(result.final_output)

asyncio.run(main())

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7d372b4b5df0>


...

LLM resp:
{
  "content": "Hello Mubashar, Welcome to Panaversity!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Hello Mubashar, Welcome to Panaversity!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Hello Mubashar, Welcome to Panaversity!



In [ ]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:
    name: str
    uid: int

@function_tool
async def greet_user(context: RunContextWrapper[UserInfo], greeting: str) -> str:
  """Greets the User with their name.
  Args:
    greeting: A specialed greeting message for user
  """
  name = context.context.name
  return f"Hello {name}, {greeting}"

async def main():
    user_info = UserInfo(name="Junaid", uid=123)

    agent = Agent[UserInfo](
        name="Assistant",
        tools=[greet_user],
        model=model,
        instructions="Always greet the user using <function_call>greet_user</function_call> and welcome them to Panaversity"
    )

    result = await Runner.run(
        starting_agent=agent,
        input="Hello",
        context=user_info,
    )

    print(result.final_output)

asyncio.run(main())

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x78b2cb02ce90>


...

LLM resp:
{
  "content": "Hello Junaid, Welcome to Panaversity!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Hello Junaid, Welcome to Panaversity!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Hello Junaid, Welcome to Panaversity!



In [21]:
from pydantic import BaseModel

from agents import Agent, handoff, RunContextWrapper

class EscalationData(BaseModel):
    reason: str

async def on_handoff(ctx: RunContextWrapper[None], input_data: EscalationData):
    print(f"Escalation agent called with reason: {input_data.reason}")

agent = Agent(name="Escalation agent",model=model)

handoff_obj = handoff(
    agent=agent,
    on_handoff=on_handoff,
    input_type=EscalationData,
)

In [28]:
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

In [25]:
async def run_escalation():
    # Await the result of the asynchronous Runner.run method
    result = await Runner.run(agent, input="escalate")
    print(result)
    print(result.final_output)

# Run the async function
asyncio.run(run_escalation())

Creating trace Agent workflow with id trace_e1900b6ae9c34e79aa9692676d4d442a


DEBUG:openai.agents:Creating trace Agent workflow with id trace_e1900b6ae9c34e79aa9692676d4d442a


Setting current trace: trace_e1900b6ae9c34e79aa9692676d4d442a


DEBUG:openai.agents:Setting current trace: trace_e1900b6ae9c34e79aa9692676d4d442a


Creating span <agents.tracing.span_data.AgentSpanData object at 0x79445d59fa70> with id None


...

LLM resp:
{
  "content": "Okay, I will try to understand the context and use of \"escalate\" you are referring to. To help me do that, could you please provide more information? For example:\n\n*   **What are you escalating?** Is it a situation, a problem, a request, a project, a task?\n*   **Who are you escalating it to?** A manager, a different department, a specific person?\n*   **Why are you escalating it?** Because you've tried to resolve it yourself and failed, because it's beyond your authority, because it's urgent?\n*   **What kind of response are you hoping for?** A solution, guidance, approval, resources?\n\nIn the meantime, here are some general meanings and potential uses of \"escalate\":\n\n**General Meanings:**\n\n*   **To increase in intensity or seriousness:** \"The conflict escalated quickly.\"\n*   **To raise to a higher level or authority:** \"I had to escalate the issue to my supervisor.\"\n*   **To increase in amount or size:** \"The price of fuel escalated dramati

DEBUG:openai.agents:LLM resp:
{
  "content": "Okay, I will try to understand the context and use of \"escalate\" you are referring to. To help me do that, could you please provide more information? For example:\n\n*   **What are you escalating?** Is it a situation, a problem, a request, a project, a task?\n*   **Who are you escalating it to?** A manager, a different department, a specific person?\n*   **Why are you escalating it?** Because you've tried to resolve it yourself and failed, because it's beyond your authority, because it's urgent?\n*   **What kind of response are you hoping for?** A solution, guidance, approval, resources?\n\nIn the meantime, here are some general meanings and potential uses of \"escalate\":\n\n**General Meanings:**\n\n*   **To increase in intensity or seriousness:** \"The conflict escalated quickly.\"\n*   **To raise to a higher level or authority:** \"I had to escalate the issue to my supervisor.\"\n*   **To increase in amount or size:** \"The price of fu

Resetting current trace


DEBUG:openai.agents:Resetting current trace


RunResult:
- Last agent: Agent(name="Escalation agent", ...)
- Final output (str):
    Okay, I will try to understand the context and use of "escalate" you are referring to. To help me do that, could you please provide more information? For example:
    
    *   **What are you escalating?** Is it a situation, a problem, a request, a project, a task?
    *   **Who are you escalating it to?** A manager, a different department, a specific person?
    *   **Why are you escalating it?** Because you've tried to resolve it yourself and failed, because it's beyond your authority, because it's urgent?
    *   **What kind of response are you hoping for?** A solution, guidance, approval, resources?
    
    In the meantime, here are some general meanings and potential uses of "escalate":
    
    **General Meanings:**
    
    *   **To increase in intensity or seriousness:** "The conflict escalated quickly."
    *   **To raise to a higher level or authority:** "I had to escalate the issue to my s

In [32]:
from agents import Agent, handoff
from agents.extensions import handoff_filters

agent = Agent(name="FAQ agent",model=model)

handoff_obj = handoff(
    agent=agent,
    input_filter=handoff_filters.remove_all_tools,
)

In [33]:
async def run_escalation():
    # Await the result of the asynchronous Runner.run method
    result = await Runner.run(agent, input="escalate")
    print(result)
    print(result.final_output)

# Run the async function
asyncio.run(run_escalation())

Creating trace Agent workflow with id trace_066b8f5ea5c54ba5ae46e84bcf73c62d
Creating trace Agent workflow with id trace_066b8f5ea5c54ba5ae46e84bcf73c62d


DEBUG:openai.agents:Creating trace Agent workflow with id trace_066b8f5ea5c54ba5ae46e84bcf73c62d


Setting current trace: trace_066b8f5ea5c54ba5ae46e84bcf73c62d
Setting current trace: trace_066b8f5ea5c54ba5ae46e84bcf73c62d


DEBUG:openai.agents:Setting current trace: trace_066b8f5ea5c54ba5ae46e84bcf73c62d


Creating span <agents.tracing.span_data.AgentSpanData object at 0x79445d5f4110> with id None
Creating span <agents.tracing.span_data.AgentSpanData object at 0x79445d5f4110> with id None


...

LLM resp:
{
  "content": "Okay, I understand you want me to \"escalate\" something. To help me do that effectively, I need more information.  What exactly do you want me to escalate? Please tell me:\n\n*   **What is the situation?** Give me the context. What's happening? What's the problem?\n*   **Who/What is involved?** Who or what are we escalating this *to*? Do you want me to escalate to a manager, a different department, a higher authority, a different system, a different service?\n*   **Why do you want to escalate it?** What's the reason for escalation? What's the desired outcome? Why is the current situation not sufficient?\n*   **What information do I have available?** What information can I use to make a good case for escalating this?\n\nFor example, you could say:\n\n*   \"Escalate this customer support ticket to a Tier 2 agent because the customer is experiencing a critical system outage and Tier 1 troubleshooting hasn't resolved the issue.\"\n*   \"Escalate this code review 

DEBUG:openai.agents:LLM resp:
{
  "content": "Okay, I understand you want me to \"escalate\" something. To help me do that effectively, I need more information.  What exactly do you want me to escalate? Please tell me:\n\n*   **What is the situation?** Give me the context. What's happening? What's the problem?\n*   **Who/What is involved?** Who or what are we escalating this *to*? Do you want me to escalate to a manager, a different department, a higher authority, a different system, a different service?\n*   **Why do you want to escalate it?** What's the reason for escalation? What's the desired outcome? Why is the current situation not sufficient?\n*   **What information do I have available?** What information can I use to make a good case for escalating this?\n\nFor example, you could say:\n\n*   \"Escalate this customer support ticket to a Tier 2 agent because the customer is experiencing a critical system outage and Tier 1 troubleshooting hasn't resolved the issue.\"\n*   \"Escala

Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


RunResult:
- Last agent: Agent(name="FAQ agent", ...)
- Final output (str):
    Okay, I understand you want me to "escalate" something. To help me do that effectively, I need more information.  What exactly do you want me to escalate? Please tell me:
    
    *   **What is the situation?** Give me the context. What's happening? What's the problem?
    *   **Who/What is involved?** Who or what are we escalating this *to*? Do you want me to escalate to a manager, a different department, a higher authority, a different system, a different service?
    *   **Why do you want to escalate it?** What's the reason for escalation? What's the desired outcome? Why is the current situation not sufficient?
    *   **What information do I have available?** What information can I use to make a good case for escalating this?
    
    For example, you could say:
    
    *   "Escalate this customer support ticket to a Tier 2 agent because the customer is experiencing a critical system outage and Tier 1 

In [35]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
)

class ScienceHomeworkOutput(BaseModel):
    is_science_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their Science homework.",
    output_type=ScienceHomeworkOutput,
    model=model
)


@input_guardrail
async def science_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_science_homework,
    )


agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[science_guardrail],
    model=model
)

async def main2():
    # This should trip the guardrail
    try:
        await Runner.run(agent, "Hello, can you help me about mitocondrea ?")
        print("Guardrail didn't trip - this is unexpected")

    except InputGuardrailTripwireTriggered:
        print("Scienc homework guardrail tripped")

In [36]:
if __name__ == "__main__":
    asyncio.run(main2())

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7d372b67a150>


...

LLM resp:
{
  "content": "{\n  \"is_science_homework\": true,\n  \"reasoning\": \"The user is asking a question about mitochondria, which is a topic commonly covered in science classes, particularly biology.\"\n}",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "{\n  \"is_science_homework\": true,\n  \"reasoning\": \"The user is asking a question about mitochondria, which is a topic commonly covered in science classes, particularly biology.\"\n}",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Math homework guardrail tripped


In [46]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail,
)
class MessageOutput(BaseModel):
    response: str

class MathOutput(BaseModel):
    reasoning: str
    is_math: bool

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the output includes any math.",
    output_type=MathOutput,
    model=model
)

@output_guardrail
async def math_guardrail(
    ctx: RunContextWrapper, customer_agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    result = await Runner.run(starting_agent=guardrail_agent, input=output.response, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math,
    )

customer_agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[math_guardrail],
    output_type=MessageOutput,
    model=model
)

async def main3():
    # This should trip the guardrail
    try:
        await Runner.run(customer_agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
        print("Guardrail didn't trip - this is unexpected")

    except OutputGuardrailTripwireTriggered:
        print("Math output guardrail tripped")

In [47]:
# prompt: generate a method asynio to run main if __name__

if __name__ == "__main__":
    asyncio.run(main3())


Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7d372b60fbf0>


...

LLM resp:
{
  "content": "{\n  \"is_math\": true,\n  \"reasoning\": \"The response includes solving an equation which involves mathematical operations and concepts.\"\n}",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "{\n  \"is_math\": true,\n  \"reasoning\": \"The response includes solving an equation which involves mathematical operations and concepts.\"\n}",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Math output guardrail tripped
